In [5]:
import random

# Define letter pair replacement rules (more complex letter combinations)
letter_pair_replacement_rules = {
    'ao': ['ae', 'oo', 'eu'],
    'ea': ['ee', 'ia', 'ae','ei'],
    'ie': ['ei', 'ae', 'ie'],
    'oo': ['oa', 'ou', 'ae'],
    'ou': ['ao', 'oo', 'ua'],
    'ae': ['ao', 'ee', 'ea'],
    'ai': ['ei', 'ae', 'ia'],
    'ui': ['oi', 'ae', 'ei']
}

# Define single letter replacement rules
letter_replacement_rules = {
    'a': ['e', 'i', 'o', 'u'],
    'b': ['p', 'v', 'f'],
    'c': ['s', 'k', 'q'],
    'd': ['t', 'r'],
    'e': ['a', 'i', 'o', 'u'],
    'f': ['v', 'b'],
    'g': ['k', 'j'],
    'h': ['g', 'k'],
    'i': ['e', 'a', 'o', 'u'],
    'j': ['g', 'y'],
    'k': ['c', 'g', 'h'],
    'l': ['r', 't'],
    'm': ['n', 'b'],
    'n': ['m', 'r'],
    'o': ['a', 'e', 'u'],
    'p': ['b', 'f', 'v'],
    'q': ['c', 'k'],
    'r': ['l', 'n', 'd'],
    's': ['c', 'z', 'x'],
    't': ['d', 'l'],
    'u': ['a', 'e', 'i', 'o'],
    'v': ['p', 'f', 'b'],
    'w': ['v', 'r'],
    'x': ['s', 'z'],
    'y': ['j', 'i'],
    'z': ['s', 'x'],
}

# Define letters that might be dropped (missing letters) and insertion words
letter_dropout = ['a', 'e', 'i', 'o', 'u', 'b', 'c', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

insertions = ['um', 'ah', 'uh', 'like', 'so']  # Insert filler words

easy_words = [
    "a", "is", "it", "an", "at", "to", "on", "up", "go", 
   "yes", "no", "in", "out", "by", "we"
]


def apply_letter_pair_replacement(word):
    """Apply letter pair replacement rules to simulate dysfluency"""
    choice_list = random.choices(list(letter_pair_replacement_rules.keys()), k = len(letter_pair_replacement_rules))
    for pair in choice_list:
        if pair in word:
            replacement = random.choice(letter_pair_replacement_rules[pair])
            word = word.replace(pair, replacement)
            break
    return word

def apply_letter_replacement(word):
    """Apply single letter replacement rules to simulate dysfluency"""
    word = list(word)
    time = 0
    while True:
        time += 1
        letter = random.choice(word)
        if letter in letter_replacement_rules:
            new_letter = random.choice(letter_replacement_rules[letter])
            word[word.index(letter)] = new_letter
            break
        if time > 10: break
    return ''.join(word)

def apply_dropout(word):
    """Simulate letter dropout (missing letters)"""
    time = 0
    choice = random.choices([1, 2, 3], weights = [0.5, 0.3, 0.2])
    if choice == 1: word = word[1:]
    elif choice == 2: word = word[:-1]
    else:
        while True:
            time += 1
            drop_letter = random.choice(letter_dropout)
            if drop_letter in word:
                word = word.replace(drop_letter, '', 1)
                break
            if time > 10: break
    return word

def simulate_dysfluency(word):
    """Simulate dysfluency on a word by applying the above rules"""
    choice = random.randint(1, 3)
    if choice == 1: word = apply_letter_pair_replacement(word)
    elif choice == 2: word = apply_letter_replacement(word)
    elif choice == 3: word = apply_dropout(word)
    return word

# Test
original_word = "please"
simulated_word = simulate_dysfluency(original_word)
print(f"Original: {original_word}, Simulated: {simulated_word}")


Original: please, Simulated: ploase


In [ ]:
import pronouncing
import numpy as np
import copy
import random
import re
import random
import nltk
from nltk.corpus import words
from tqdm import tqdm

TYPE = ("REP","SUB","INS","POLY","DEL")

nltk.download('words')

word_list = words.words()

SENT = []
REF = []
SRC = []
LABEL = []
NUM = []

with open('pre_VCTK.txt', 'r', encoding='utf-8') as file:
    total_lines = sum(1 for _ in file)
    file.seek(0)
    for line in tqdm(file, total = total_lines * 3, desc = "Line deal"):
        sentence = line.strip()
        if "should be doing that for herThey have" in sentence or "falls upon the green of the first" in sentence or "It is up to Ann" in sentence:
            continue
        ref_word = sentence.lower().split(" ")
        idx = list(range(len(ref_word)))
        if len(ref_word) < 7: 
            generate_time = random.randint(1, 4)
        elif len(ref_word) >= 7 and len(ref_word) < 10:
            generate_time = random.randint(12, 15)
        elif len(ref_word) >= 10:
            generate_time = random.randint(15, 20)
        for t in range(generate_time):
            src_word = copy.deepcopy(ref_word)
            label = [1] * len(src_word)
            if len(ref_word) < 7:
                dysfluent_times = random.randint(0, 1)
            elif len(ref_word) >= 7 and len(ref_word) < 10:
                dysfluent_times = random.randint(1, 3)
            elif len(ref_word) >= 10:
                dysfluent_times = random.randint(1, 4)
            dysfluent_pb = []
            for i in ref_word:
                if i in easy_words:
                    dysfluent_pb.append(0.01)
                else:
                    dysfluent_pb.append(0.5)
            sample_result = np.random.choice(idx, size = dysfluent_times, replace = False, p = dysfluent_pb/np.sum(dysfluent_pb))
            sample_result = np.sort(sample_result)
            dysfluent_word = np.array(ref_word)[sample_result]
            for j in range(len(sample_result)):
                target_word = dysfluent_word[j]
                target_idx = sample_result[j]
                dysfluent_type = random.choices(TYPE, weights = [0.25, 0.5, 0.4, 0.25, 0.4], k = 1)[0]

                if dysfluent_type == "POLY":
                    flg = False
                    if j == len(sample_result) - 1:
                        if target_idx + 1 < len(src_word): flg = True
                    else:
                        if sample_result[j + 1] - target_idx != 1: flg = True
                    if flg:
                        src_word.insert(target_idx, src_word[target_idx + 1])
                        src_word.insert(target_idx, target_word)
                        label.insert(target_idx, 0)
                        label.insert(target_idx, 0)
                        for m in range(len(sample_result)):
                            if sample_result[m] >= target_idx: sample_result[m] += 2
                    else: dysfluent_type = "REP"

                if dysfluent_type == "REP":
                    repetition_time = random.randint(1, 2)
                    for k in range(repetition_time):
                        src_word.insert(target_idx, target_word)
                        label.insert(target_idx, 0)
                    for m in range(len(sample_result)):
                        if sample_result[m] >= target_idx: sample_result[m] += repetition_time

                elif dysfluent_type == "SUB":
                    src_word[target_idx] = simulate_dysfluency(target_word)

                elif dysfluent_type == "DEL":
                    label[target_idx] = 2
                    src_word[target_idx] = "[None]"

                elif dysfluent_type == "INS":
                    ins_word_1 = random.choices(insertions, weights = [0.3, 0.3, 0.3, 0.2, 0.2])[0]
                    ins_word_2 = random.choices(insertions, weights = [0.3, 0.3, 0.3, 0.2, 0.2])[0]
                    rand_word_1 =  random.choice(word_list)
                    rand_word_2 = random.choice(word_list)
                    multi = random.choices(population=[1, 2, 3, 4],  weights = [0.8, 0, 0.2, 0])[0]
                    if multi == 1:
                        flg = random.randint(0, 1)
                        if flg:
                            src_word.insert(target_idx, ins_word_1)
                        else:
                            src_word.insert(target_idx + 1, ins_word_1)
                        label.insert(target_idx, 0)
                        for m in range(len(sample_result)):
                            if sample_result[m] > target_idx: sample_result[m] += 1
                    elif multi == 2:
                        flg = random.randint(0, 1)
                        if flg:
                            src_word.insert(target_idx, rand_word_1)
                        else:
                            src_word.insert(target_idx + 1, rand_word_1)
                        label.insert(target_idx, 0)
                        for m in range(len(sample_result)):
                            if sample_result[m] > target_idx: sample_result[m] += 1
                    elif multi == 3:
                        flg = random.randint(0, 3)
                        if flg == 0:
                            src_word.insert(target_idx, ins_word_1)
                            src_word.insert(target_idx, ins_word_2)
                        elif flg == 1:
                            src_word.insert(target_idx, ins_word_1)
                            src_word.insert(target_idx, target_word)
                        elif flg == 2:
                            src_word.insert(target_idx, target_word)
                            src_word.insert(target_idx, ins_word_1)
                        elif flg == 3:
                            src_word.insert(target_idx + 1, ins_word_1)
                            src_word.insert(target_idx + 1, ins_word_2)
                        label.insert(target_idx, 0)
                        label.insert(target_idx, 0)
                        for m in range(len(sample_result)):
                            if sample_result[m] > target_idx: sample_result[m] += 2
                    elif multi == 4:
                        flg = random.randint(0, 3)
                        if flg == 0:
                            src_word.insert(target_idx, rand_word_1)
                            src_word.insert(target_idx, rand_word_2)
                        elif flg == 1:
                            src_word.insert(target_idx, rand_word_1)
                            src_word.insert(target_idx, target_word)
                        elif flg == 2:
                            src_word.insert(target_idx, target_word)
                            src_word.insert(target_idx, rand_word_1)
                        elif flg == 3:
                            src_word.insert(target_idx + 1, rand_word_1)
                            src_word.insert(target_idx + 1, rand_word_2)
                        label.insert(target_idx, 0)
                        label.insert(target_idx, 0)
                        for m in range(len(sample_result)):
                            if sample_result[m] > target_idx: sample_result[m] += 2

            REF.append(" ".join(ref_word))
            src_word = list(filter(lambda x: x != "[None]", src_word))
            SRC.append(" ".join(src_word))
            label.append(3)
            LABEL.append(" ".join([str(i) for i in label]))
            NUM.append(dysfluent_times)

[nltk_data] Downloading package words to /home/zongli/nltk_data...
[nltk_data]   Package words is already up-to-date!
Line deal:   0%|          | 70/300000 [00:00<07:12, 692.76it/s]

Line deal:  33%|███▎      | 100000/300000 [02:30<05:01, 662.85it/s]


In [10]:
import json

data = []

for idx, (ref, src, label, num) in enumerate(zip(REF, SRC, LABEL, NUM)):
    data.append({
        "id": idx+1,
        "REF": ref,
        "SRC": src,
        "LABEL": label,
        "num": num
    })

with open('train_data/word/data5.json', 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

In [11]:
# Preview
import json
with open('train_data/word/data5.json', 'r') as file:
    data = json.load(file)
    print(len(data))
    print(data[-5:])

1125034
[{'id': 1125030, 'REF': 'after this there is a long period with virtually nothing surviving', 'SRC': 'after this um there is a long long long period with virtually nothing surviving', 'LABEL': '1 0 1 1 1 1 0 0 1 1 1 1 1 1 3', 'num': 2}, {'id': 1125031, 'REF': 'after this there is a long period with virtually nothing surviving', 'SRC': 'after this after this there is a long uh long period wit virtually nothing surviving', 'LABEL': '0 0 1 1 1 1 1 0 0 1 1 1 1 1 1 3', 'num': 3}, {'id': 1125032, 'REF': 'after this there is a long period with virtually nothing surviving', 'SRC': 'after this there is a long period with virtually nothing nothing nothing surviving', 'LABEL': '1 1 1 1 1 1 1 1 1 0 0 1 1 3', 'num': 2}, {'id': 1125033, 'REF': 'after this there is a long period with virtually nothing surviving', 'SRC': 'after this there is a um period period with virtually', 'LABEL': '1 1 1 1 1 2 0 0 1 1 1 2 2 3', 'num': 4}, {'id': 1125034, 'REF': 'after this there is a long period with virt

after this there is a long period with virtually nothing surviving
after this um there is a long long long period with virtually nothing surviving